<a href="https://colab.research.google.com/github/xiaoyi-yuxiao/MachineLearningTheory/blob/master/machine_learning_linear_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=c05dcf9fe6818c14afd3f59a7237244b12807f0c7cd772dd219d241def816f15
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
class ML_OLS:

  def __init__(self, X, Y):

    self.__X,self.__Y = self.handle_data(X,Y)
    self.__diff = 0

  def handle_data(self,X,Y):
    
    #convert pandas to numpy
    X = X.to_numpy()
    Y = Y.to_numpy()
    #reshape the data to form a matrix
    X_reshaped = X.reshape((len(X),1))
    Y_reshaped = Y.reshape((len(Y),1))
    X_reshaped = (X_reshaped-X_reshaped.mean())/np.std(X_reshaped)
    Y_reshaped = (Y_reshaped-Y_reshaped.mean())/np.std(Y_reshaped)

    #add ones to X_reshaped
    add_ones = np.ones([len(X),1])
    Appended_X = np.append(add_ones,X_reshaped,axis=1)

    return Appended_X,Y_reshaped

  def derivative(self,theta_0,theta_1):

    
    #put theta_0 and theta_1 into a data array with shape
    theta = np.array([theta_0,theta_1])

    #calculate hypothesis reshape hypothesis
    h = np.dot(self.__X, theta)
    reshaped_h = h.reshape((len(h),1))

    #calculate difference between Y and hypothesis
    self.__diff = self.__Y - reshaped_h

    #calculate the derivative
    deri_theta_0 = (1/len(self.__X)) * sum(self.__Y - reshaped_h)
    deri_theta_1 = (1/len(self.__X)) * np.dot(self.__diff.transpose(), self.__X[:,1])
    
    return deri_theta_0, deri_theta_1

  def compute_cost(self):

    cost = 1/(2*len(self.__diff))*np.dot(self.__diff.transpose(),self.__diff)
    return cost

  def update(self, learning_rate,theta_0,theta_1):

    deri_theta_0, deri_theta_1 = self.derivative(theta_0,theta_1)
    theta_0 = theta_0 + learning_rate * deri_theta_0
    theta_1 = theta_1 + learning_rate * deri_theta_1
    return theta_0,theta_1  


  def summury(self, learning_rate):

    #set initial
    theta_0,theta_1 = 0,0

    stop_condition = 0.05
    #interation times
    i = 1

    #interating to find minimum derivative for theta0 and theta1
    while True:

      deri_theta_0,deri_theta_1 = self.derivative(theta_0,theta_1)
      print("Cost on interation" + str(i) + " is " + str(self.compute_cost()))
      print(deri_theta_1+deri_theta_0)
      
      if (deri_theta_1+deri_theta_0) < stop_condition or i >1000:
        break
      else:
      #updating interating times each interation
        theta_0,theta_1 = self.update(learning_rate,theta_0,theta_1)

      #add interating times
      i = i+1
    return theta_0,theta_1

  def get_XY(self):
    return self.__X,self.__Y

In [ ]:
start = '2012-12-12'
end = '2013-12-30'
Y = yf.download('AAPL',start=start,end=end,interval='1d')['Adj Close']
X = yf.download('^GSPC',start=start,end=end,interval='1d')['Adj Close']
ret_x = X.pct_change()[1:]
ret_y = Y.pct_change()[1:]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
test_case = ML_OLS(X,Y)

In [ ]:
a,b = test_case.summury(0.05)

Cost on interation1 is [[0.5]]
[0.43013163]
Cost on interation2 is [[0.49098061]]
[0.40862505]
Cost on interation3 is [[0.4828406]]
[0.3881938]
Cost on interation4 is [[0.47549425]]
[0.36878411]
Cost on interation5 is [[0.46886417]]
[0.3503449]
Cost on interation6 is [[0.46288051]]
[0.33282766]
Cost on interation7 is [[0.45748027]]
[0.31618627]
Cost on interation8 is [[0.45260655]]
[0.30037696]
Cost on interation9 is [[0.44820802]]
[0.28535811]
Cost on interation10 is [[0.44423834]]
[0.27109021]
Cost on interation11 is [[0.44065571]]
[0.2575357]
Cost on interation12 is [[0.43742238]]
[0.24465891]
Cost on interation13 is [[0.43450431]]
[0.23242596]
Cost on interation14 is [[0.43187074]]
[0.22080467]
Cost on interation15 is [[0.42949395]]
[0.20976443]
Cost on interation16 is [[0.42734889]]
[0.19927621]
Cost on interation17 is [[0.42541298]]
[0.1893124]
Cost on interation18 is [[0.42366582]]
[0.17984678]
Cost on interation19 is [[0.42208901]]
[0.17085444]
Cost on interation20 is [[0.42066

In [ ]:
plt.scatter(X[:,1],Y)

In [ ]:
ret_Mc = sm.add_constant(X_reshaped)
results = sm.OLS(Y_reshaped,ret_Mc).fit()
print(results.summary())